In [451]:
import uuid
import maya
import orjson
from copy import copy
from redis import Redis
from typing import List, Set
from jamboree.utils.helper import Helpers

In [452]:
redis = Redis()
helper = Helpers()

In [453]:
def add_time(item:dict, _time:float, rel_abs="absolute"):
    if rel_abs == "absolute":
        item['timestamp'] = _time
    else:
        item['time'] = _time
    return item

In [454]:
def generate_dicts(data, _time, timestamp):
    relative = copy(data)
    absolute = copy(data)
    relative['time'] = _time
    absolute['timestamp'] = _time
    return {
        "relative": relative,
        "absolute": absolute
    }

In [455]:
def sorted_z_to_dict(zset:List[Set], rel_abs="absolute"):
    if len(zset) == 0 or abs_rel not in ["absolute", "relative"]:
        return []
    
    times = [x[1] for x in zset]
    dicts = [add_time(orjson.loads(x[0]), times[i], rel_abs) for i, x in enumerate(zset)]
    return dicts

In [475]:
def dictify(azset:List[Set], rzset:List[Set]):
    """Create a dictionary"""
    if len(azset) == 0 or len(rzset) == 0:
        return {}
    adict = {}
    for azs in azset:
        item, time = azs
        if item == b'{"placeholder": "place"}':
            continue
        current_item = adict.get(item, {})
        current_item['timestamp'] = time
        adict[item] = current_item
    
    # Set the relative time
    for rzs in rzset:
        item, time = rzs
        if item == b'{"placeholder": "place"}':
            continue
        current_item = adict.get(item, {})
        current_item['time'] = time
        adict[item] = current_item
    
    return adict

In [476]:
def deserialize_dicts(dictified:dict):
    _deserialized = []
    for key, value in dictified.items():
        _key = orjson.loads(key)
        _key['time'] = value.get("time", maya.now()._epoch)
        _key['timestamp'] = value.get("timestamp", maya.now()._epoch)
        _deserialized.append(_key)
    return _deserialized

In [477]:
def check_time(_time:float, _timestamp:float, local_time:float, local_timestamp:float):
    current_time = maya.now()._epoch
    
    
    if local_time is not None:
        _time = local_time
    elif _time is None:
        _time = current_time
    if local_timestamp is not None:
        _timestamp = local_timestamp
    elif _timestamp is None:
        _timestamp = current_time
    
    return {
        "time": _time,
        "timestamp": _timestamp
    }

In [478]:
def separate_time_data(data:dict, _time:float=None, _timestamp:float=None):
    local_time = data.pop("time", None)
    local_timestamp = data.pop("timestamp", None)
    timing = check_time(_time, _timestamp, local_time, local_timestamp)
    return data, timing

In [479]:
def save(query, data, _time=None, _timestamp=None):
    if not helper.validate_query(query):
        return 
    _hash = helper.generate_hash(query)
    
    
    query.update(data)
    
    
    
    data, timing = separate_time_data(query, _time, _timestamp)
    
    relative_time_key = f"{_hash}:rlist"
    absolute_time_key = f"{_hash}:alist"

    
    # Generate Data
    mono = orjson.dumps(data)
    relative_data = {
        mono: timing["time"]
    }
    absolute_data = {
        mono: timing["timestamp"]
    }    

    redis.zadd(relative_time_key, relative_data)
    redis.zadd(absolute_time_key, absolute_data)

In [480]:
def query(_query):
    if not helper.validate_query(_query):
        return 
    _hash = helper.generate_hash(_query)
    relative_time_key = f"{_hash}:rlist"
    absolute_time_key = f"{_hash}:alist"
    keys = redis.zrange(relative_time_key, 0, -1, withscores=True)
    akeys = redis.zrange(absolute_time_key, 0, -1, withscores=True)

    dicts = dictify(akeys, keys)
    combined = deserialize_dicts(dicts)
    return combined

In [481]:
def query_latest(_query, abs_rel="absolute"):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    blank_keys = [(b'{"placeholder": "place"}', 0)]
    keys = redis.zrange(_current_key, -1, -1, withscores=True)
    dicts = dictify(keys, blank_keys)
    combined = deserialize_dicts(dicts)
    return combined

In [482]:
def query_latest_many(_query, abs_rel="absolute", limit:int=10):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    blank_keys = [(b'{"placeholder": "place"}', 0)]
    keys = redis.zrange(_current_key, -limit, -1, withscores=True)
    dicts = []
    if abs_rel == "absolute":
        dicts = dictify(keys, blank_keys)
    else:
        
        dicts = dictify(blank_keys, keys)
    
    combined = deserialize_dicts(dicts)
    return combined

In [483]:
episode = uuid.uuid1().hex

In [484]:
save({"type": "hello", "episode": episode}, {"name":"world"}, _time=(maya.now()._epoch + 3600))
save({"type": "hello", "episode": episode}, {"name":"world", "my": "world"}, _time=(maya.now()._epoch))

In [485]:
query_latest({"type": "hello", "episode": episode}, abs_rel="relative")

[{'type': 'hello',
  'episode': '78691f4e33d711ea9d1380c5f21e8205',
  'name': 'world',
  'time': 1578681112.3724682,
  'timestamp': 1578684712.0971866}]

In [486]:
query_latest({"type": "hello", "episode": episode})

[{'type': 'hello',
  'episode': '78691f4e33d711ea9d1380c5f21e8205',
  'name': 'world',
  'my': 'world',
  'time': 1578681113.140772,
  'timestamp': 1578681112.0989897}]

In [487]:
query_latest_many({"type": "hello", "episode": episode}, abs_rel="relative", limit=100)

[{'type': 'hello',
  'episode': '78691f4e33d711ea9d1380c5f21e8205',
  'name': 'world',
  'my': 'world',
  'time': 1578681112.0989506,
  'timestamp': 1578681114.2134943},
 {'type': 'hello',
  'episode': '78691f4e33d711ea9d1380c5f21e8205',
  'name': 'world',
  'time': 1578684712.0971866,
  'timestamp': 1578681114.2135043}]